<a href="https://colab.research.google.com/github/Dharuva/Algo-Trading-Scripts-/blob/main/ML_KNN_indicator_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=9dfc8554cc504b47750d66d7d591ac0dc7403bce63ce06d1cedebe6bce394247
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
from backtesting.test import SMA
import pandas as pd
import numpy as np
from pandas_datareader.data import DataReader
import matplotlib.pyplot as plt
data = pd.read_csv('/content/CleanData2.csv', parse_dates=True, index_col='Date')
data

,Open,High,Low,Close
Date,,,,
2020-01-01 09:16:00,30847.72637,30908.17490,30802.26027,30806.38801
2020-01-01 09:17:00,30809.99476,30822.24503,30801.45383,30819.20530
2020-01-01 09:18:00,30819.20530,30822.05503,30814.74177,30821.39004
2020-01-01 09:19:00,30821.39004,30831.22525,30803.39885,30804.67985
2020-01-01 09:20:00,30806.43546,30810.75419,30793.86586,30802.49747
...,...,...,...,...
2022-12-30 15:26:00,23110.54002,23144.45146,23108.56403,23135.66988
2022-12-30 15:27:00,23133.20795,23134.09097,23110.70025,23111.47473
2022-12-30 15:28:00,23111.82193,23121.38728,23104.82660,23121.38728


In [ ]:
def BBANDS(data, n_lookback, n_std):
    """Bollinger bands indicator"""
    hlc3 = (data.High + data.Low + data.Close) / 3
    mean, std = hlc3.rolling(n_lookback).mean(), hlc3.rolling(n_lookback).std()
    upper = mean + n_std*std
    lower = mean - n_std*std
    return upper, lower

close = data.Close.values
sma10 = SMA(data.Close, 10)
sma20 = SMA(data.Close, 20)
sma50 = SMA(data.Close, 50)
sma100 = SMA(data.Close, 100)
upper, lower = BBANDS(data, 20, 2)

# Design matrix / independent features:

# Price-derived features
data['X_SMA10'] = (close - sma10) / close
data['X_SMA20'] = (close - sma20) / close
data['X_SMA50'] = (close - sma50) / close
data['X_SMA100'] = (close - sma100) / close

data['X_DELTA_SMA10'] = (sma10 - sma20) / close
data['X_DELTA_SMA20'] = (sma20 - sma50) / close
data['X_DELTA_SMA50'] = (sma50 - sma100) / close

# Indicator features
data['X_MOM'] = data.Close.pct_change(periods=2)
data['X_BB_upper'] = (upper - close) / close
data['X_BB_lower'] = (lower - close) / close
data['X_BB_width'] = (upper - lower) / close
data['X_Sentiment'] = ~data.index.to_series().between('2020-01-01', '2020-03-26')


# Some datetime features for good measure
data['X_day'] = data.index.dayofweek
data['X_hour'] = data.index.hour


data = data.dropna().astype(float)

In [ ]:
data

,Open,High,Low,Close,X_SMA10,X_SMA20,X_SMA50,X_SMA100,X_DELTA_SMA10,X_DELTA_SMA20,X_DELTA_SMA50,X_MOM,X_BB_upper,X_BB_lower,X_BB_width,X_Sentiment,X_day,X_hour
Date,,,,,,,,,,,,,,,,,,
2020-01-01 10:55:00,30996.12239,30998.09207,30978.07062,30980.32594,0.000454,0.000806,0.001072,0.002683,0.000352,0.000266,0.001611,-0.000203,0.000252,-0.001853,0.002104,0.0,2.0,10.0
2020-01-01 10:56:00,30981.42973,30988.53424,30980.85383,30982.72558,0.000408,0.000826,0.001108,0.002704,0.000418,0.000282,0.001595,-0.000418,0.000301,-0.001934,0.002236,0.0,2.0,10.0
2020-01-01 10:57:00,30983.92554,30986.56577,30976.48730,30977.63879,0.000151,0.000607,0.000906,0.002489,0.000456,0.000299,0.001583,-0.000087,0.000552,-0.001744,0.002296,0.0,2.0,10.0
2020-01-01 10:58:00,30978.83836,30981.33375,30974.28051,30978.93432,0.000116,0.000599,0.000906,0.002480,0.000484,0.000307,0.001574,-0.000122,0.000577,-0.001753,0.002330,0.0,2.0,10.0
2020-01-01 10:59:00,30977.39889,30983.73354,30975.81563,30979.55814,0.000072,0.000573,0.000886,0.002443,0.000501,0.000312,0.001558,0.000062,0.000623,-0.001742,0.002364,0.0,2.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 15:26:00,23110.54002,23144.45146,23108.56403,23135.66988,-0.000821,-0.001253,0.000459,0.000823,-0.000432,0.001712,0.000364,0.001196,0.003292,-0.000768,0.004060,1.0,4.0,15.0
2022-12-30 15:27:00,23133.20795,23134.09097,23110.70025,23111.47473,-0.001548,-0.002134,-0.000589,-0.000227,-0.000586,0.001545,0.000362,0.000042,0.004227,0.000044,0.004183,1.0,4.0,15.0
2022-12-30 15:28:00,23111.82193,23121.38728,23104.82660,23121.38728,-0.000871,-0.001559,-0.000172,0.000195,-0.000688,0.001386,0.000367,-0.000617,0.003788,-0.000655,0.004442,1.0,4.0,15.0


In [ ]:
import numpy as np


def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

In [ ]:
get_y(data).dropna()

Date
2020-01-01 10:55:00    0.0
2020-01-01 10:56:00    0.0
2020-01-01 10:57:00    0.0
2020-01-01 10:58:00    0.0
2020-01-01 10:59:00    0.0
                      ... 
2021-10-21 12:50:00    0.0
2021-10-21 12:51:00    0.0
2021-10-21 12:52:00    0.0
2021-10-21 12:53:00    0.0
2021-10-21 12:54:00   -1.0
Name: Close, Length: 168120, dtype: float64

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X, y = get_clean_Xy(data)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.8, random_state=1) # not correct for trading data

#correct
train_index= int(0.8*len(X)) # for taking 80% of data for testing
X_train,X_test = X[:train_index],X[train_index:]
y_train,y_test = y[:train_index],y[train_index:]

clf = KNeighborsClassifier(n_neighbors=6)
clf.fit(X_train, y_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

# _ = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).plot(figsize=(30, 2), alpha=.7)
# print('Classification accuracy: ', np.mean(y_test == y_pred))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_train = accuracy_score(y_train,y_pred_train)
accuracy_test = accuracy_score(y_test,y_pred_test)
print("Accuracy train : %.2f%%" % (accuracy_train*100))
print("Accuracy test : %.2f%%" % (accuracy_test*100))

print(get_y(data).dropna().value_counts()*100/get_y(data).dropna().count())

#gambler test

pred_test = np.random.choice([0,-1,1],len(y_pred_test))
accuracy_test2 =  accuracy_score(y_test,pred_test)
print("Accuracy Gambler : %.2f%%" % (accuracy_test2*100))

Accuracy train : 89.20%
Accuracy test : 39.70%
 0.0    54.992382
 1.0    22.731517
-1.0    22.276100
Name: Close, dtype: float64
Accuracy Gambler : 33.11%


In [ ]:
#ML + SMA
from backtesting import Backtest, Strategy

N_TRAIN = 222875






date_array = [20200130, 20200227, 20200326, 20200430, 20200528, 20200625,
              20200730, 20200827, 20200924, 20201029, 20201126, 20201231,
              20210128, 20210225, 20210325, 20210429, 20210527, 20210624,
              20210729, 20210826, 20210930, 20211028, 20211125, 20211230,
              20220127, 20220224, 20220331, 20220428, 20220526, 20220630,
              20220728, 20220825, 20220929, 20221027, 20221124, 20221229,
              20230125]


timestamp_array = pd.to_datetime(date_array, format='%Y%m%d') + pd.Timedelta('15:30:00')




class TrailingStrategy(Strategy):

    __n_atr = 6.
    __atr = None

    def init(self):
        super().init()
        self.set_atr_periods()

    def set_atr_periods(self, periods: int = 14):

        hi, lo, c_prev = self.data.High, self.data.Low, pd.Series(self.data.Close).shift(1)
        tr = np.max([hi - lo, (c_prev - hi).abs(), (c_prev - lo).abs()], axis=0)
        atr = pd.Series(tr).rolling(periods).mean().bfill().values
        self.__atr = atr

    def set_trailing_sl(self, n_atr: float = 6):
        self.__n_atr = n_atr

    def next(self):
        super().next()
        index = len(self.data)-1
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf,
                               self.data.Close[index] - self.__atr[index] * self.__n_atr)
            else:
                trade.sl = min(trade.sl or np.inf,
                               self.data.Close[index] + self.__atr[index] * self.__n_atr)







class MLTrainOnceStrategy(TrailingStrategy):

    price_delta = .004  # 0.4%
    i_ma1 = 225
    i_ma2 = 5
    stop_loss_percent = 0.09
    prevBuyPrice = None

    timestamp_array = pd.to_datetime(date_array, format='%Y%m%d') + pd.Timedelta('15:30:00')

    buyPrice = 0

    stopNunu =40

    def init(self):

        super().init()
        super().set_trailing_sl(7)


        # our sma strategy indicator

        self.ma1 = self.I(SMA, self.data.Close, self.i_ma1)
        self.ma2 = self.I(SMA, self.data.Close, self.i_ma2)


        # Init our model, a kNN classifier
        self.clf = KNeighborsClassifier(300)

        # Train the classifier in advance on the first N_TRAIN examples
        df = self.data.df.iloc[:N_TRAIN]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):
        #Skip the training, in-sample data
        # if len(self.data) < N_TRAIN:
        #     return

        super().next()

        expiryCondition = (self.data.index[-1] in timestamp_array)

        buyCondition = (self.data.Close[-1] > self.ma1[-1]) and (self.data.Close[-1] < self.ma2[-1]) and (not self.position)

        buyclose = (self.ma2[-1]<self.data.Close[-1]) and (self.position) and (self.data.Close[-1] < self.data.Low[-2])

        sellCondition = (self.data.Close[-1] < self.ma1[-1]) and (self.data.Close[-1] > self.ma2[-1]) and (not self.position)

        sellclose = (self.ma2[-1]>self.data.Close[-1]) and (self.position) and (self.data.Close[-1] > self.data.High[-2])

        price = self.data.Close[-1]

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.


        # upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)

        # if forecast == 1 and not self.position.is_long:
        #     self.buy( size=100,tp=upper, sl=lower)
        # elif forecast == -1 and not self.position.is_short:
        #     self.sell(size=100, tp=lower, sl=upper)




        if buyCondition and forecast > 0.3 and not expiryCondition  and not self.position.is_long:
            self.buy(size=100,sl=price-self.stopNunu)
            self.buyPrice = self.data.Open[-1]


        if buyclose or expiryCondition:
            self.position.close()
            buyPrice = 0

        if sellCondition and forecast <-0.3  and not expiryCondition and not self.position.is_short :
            self.sell(size=100,sl=price+self.stopNunu)
            self.buyPrice = self.data.Open[-1]


        if sellclose or expiryCondition :
            self.position.close()
            buyPrice = 0


        # Additionally, set aggressive stop-loss on trades that have been open
        # for more than two days
        # for trade in self.trades:
        #     if current_time - trade.entry_time > pd.Timedelta('2 days'):
        #         if trade.is_long:
        #             trade.sl = max(trade.sl, low)
        #         else:
        #             trade.sl = min(trade.sl, high)

#data = pd.read_csv('/content/CleanData2.csv', parse_dates=True, index_col='Date')
bt = Backtest(data, MLTrainOnceStrategy, commission=.000105,cash=50_00_000)
stats=bt.run()
bt.plot()
print(stats)

Streaming output truncated to the last 5000 lines.
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in

Start                     2020-01-01 10:55:00
End                       2022-12-30 15:30:00
Duration                   1094 days 04:35:00
Exposure Time [%]                   23.816672
Equity Final [$]               6983445.467798
Equity Peak [$]                7879018.357943
Return [%]                          39.668909
Buy & Hold Return [%]              -25.337563
Return (Ann.) [%]                   12.032921
Volatility (Ann.) [%]               11.945804
Sharpe Ratio                         1.007293
Sortino Ratio                        2.318363
Calmar Ratio                          1.00634
Max. Drawdown [%]                  -11.957119
Avg. Drawdown [%]                   -0.524883
Max. Drawdown Duration      927 days 02:12:00
Avg. Drawdown Duration        8 days 00:34:00
# Trades                                 4330
Win Rate [%]                        26.004619
Best Trade [%]                      11.626519
Worst Trade [%]                     -4.345077
Avg. Trade [%]                    